# Search Algorithms and Probability Concepts

This notebook demonstrates various search algorithms (BFS, DFS, UCS, A*) used in artificial intelligence for finding paths in graphs. Each algorithm has different properties in terms of completeness, optimality, and computational efficiency.

## Conditional Probability Formula

$$
P(A=a \mid b) \;=\; \frac{P(a,b)}{P(b)} \;=\; \frac{P(a,b)}{\sum_{x \in A} P(x,b)}
$$

The formula above represents **Conditional Probability**, which calculates the probability of event A=a given that event B=b has occurred. It is calculated by:

1. Dividing the joint probability P(a,b) of both events occurring by the marginal probability P(b) of event B=b occurring
2. The marginal probability P(b) can be calculated by summing the joint probabilities over all possible values of A

This formula is fundamental in Bayesian probability theory and is used in many AI applications including Bayesian networks and probabilistic reasoning.

In [ ]:
from collections import deque, defaultdict
G = defaultdict(list)
G['A'] = ['C','B']; G['C'] = ['B','D']; G['B'] = ['D','G']; G['D'] = ['G']
def bfs(s, t):
    q = deque([s]); parent = {s: None}
    while q:
        u = q.popleft()
        if u == t: break
        for v in G[u]:
            if v not in parent:
                parent[v] = u; q.append(v)
    path = []
    if t in parent:
        cur = t
        while cur is not None: path.append(cur); cur = parent[cur]
        path.reverse()
    return path
print(bfs('A','G'))  # ['A','B','G']

## Breadth-First Search (BFS) Algorithm

BFS explores a graph by visiting all neighbors of a node before moving to the next level neighbors. It uses a queue data structure (First-In-First-Out) to keep track of nodes to visit.

**Characteristics:**
- **Completeness:** Yes - will find a solution if one exists (assuming finite branching factor)
- **Optimality:** Yes - finds the shortest path in terms of number of edges (when all edges have equal cost)
- **Time Complexity:** O(b^d) where b is the branching factor and d is the depth of the solution
- **Space Complexity:** O(b^d) as it must store all nodes at the frontier

**Algorithm Steps:**
1. Initialize a queue with the start node and a dictionary to track parent nodes
2. Until the queue is empty or goal is found:
   - Dequeue a node
   - If it's the goal node, reconstruct the path and return it
   - Otherwise, enqueue all unvisited neighbors and record their parent

**Example Graph:**
```
A --- B --- G
|     |
C --- D
```

The BFS will explore nodes in the order: A, C, B, D, G

In [ ]:
path = []
def dfs(u, t, seen=set()):
    if u==t: return [t]
    seen.add(u)
    for v in G[u]:
        if v not in seen:
            p = dfs(v,t,seen)
            if p: return [u]+p
    return []
print(dfs('A','G',set()))  # ['A','C','D','G']

## Depth-First Search (DFS) Algorithm

DFS explores a graph by going as deep as possible along each branch before backtracking. It uses recursion or a stack data structure (Last-In-First-Out) to keep track of nodes to visit.

**Characteristics:**
- **Completeness:** Yes - will find a solution if one exists (assuming finite depth), but may not terminate on infinite graphs
- **Optimality:** No - doesn't guarantee the shortest path
- **Time Complexity:** O(b^m) where b is the branching factor and m is the maximum depth
- **Space Complexity:** O(m) - only needs to store the path from root to current node plus unexplored siblings

**Algorithm Steps:**
1. Mark the current node as visited
2. If the current node is the goal, return the path
3. Otherwise, recursively visit each unvisited neighbor
4. If no path is found after exploring all neighbors, backtrack

**Example Graph:**
```
A --- B --- G
|     |
C --- D
```

The DFS might explore nodes in the order: A, C, D, G (if it chooses C first) or A, B, G (if it chooses B first)

In [ ]:
import heapq
W = {('A','C'):1, ('A','B'):3, ('C','B'):1, ('C','D'):3, ('B','D'):1, ('B','G'):3, ('D','G'):1}
def ucs(s,t):
    pq=[(0,s,None)]; best={s:0}; parent={}
    while pq:
        g,u,p = heapq.heappop(pq)
        if u in parent:  # already finalized
            continue
        parent[u]=p
        if u==t: break
        for v in G[u]:
            w=W[(u,v)]
            if v not in best or g+w<best[v]:
                best[v]=g+w
                heapq.heappush(pq,(g+w,v,u))
    # rebuild
    path=[]; u=t
    while u is not None: path.append(u); u=parent.get(u)
    path.reverse()
    return path, best[t]
print(ucs('A','G'))  # (['A','C','B','D','G'], 4)

## Uniform Cost Search (UCS) Algorithm

UCS is a variant of Dijkstra's algorithm that explores a graph by always expanding the node with the lowest cumulative path cost. It uses a priority queue to select the next node to explore.

**Characteristics:**
- **Completeness:** Yes - will find a solution if one exists (assuming positive edge costs)
- **Optimality:** Yes - finds the lowest-cost path (assuming positive edge costs)
- **Time Complexity:** O(b^(C*/ε)) where C* is the cost of the optimal solution and ε is the minimum edge cost
- **Space Complexity:** O(b^(C*/ε)) as it must store all nodes in the frontier

**Algorithm Steps:**
1. Initialize a priority queue with the start node (priority 0)
2. Until the queue is empty or goal is found:
   - Dequeue the node with lowest cumulative cost
   - If it's the goal node, reconstruct the path and return it
   - Otherwise, enqueue all neighbors with their cumulative costs as priority

**Edge Weights Table:**
| Edge | Cost |
|------|------|
| A-C  | 1    |
| A-B  | 3    |
| C-B  | 1    |
| C-D  | 3    |
| B-D  | 1    |
| B-G  | 3    |
| D-G  | 1    |

**Example Graph with Edge Costs:**
```
A --3-- B --3-- G
|      /|
1     1 |
|    /  |
C --3-- D --1-- G
```

UCS will find the optimal path A→C→B→D→G with total cost 4

| Node | h(n) |
| ---- | ---- |
| A    | 3    |
| B    | 2    |
| C    | 2    |
| D    | 1    |
| G    | 0    |


## Heuristic Function

The table above shows the heuristic function h(n) for each node, which estimates the cost from that node to the goal G. A good heuristic should:

1. Be admissible - never overestimate the true cost to reach the goal
2. Be consistent - satisfy the triangle inequality (h(n) ≤ c(n,n') + h(n'))

This particular heuristic represents the "straight-line distance" or "direct distance" to the goal node G. Notice that h(G) = 0 since the cost from G to itself is 0.

In [ ]:
import heapq
G = {'A':['C','B'],'C':['B','D'],'B':['D','G'],'D':['G'],'G':[]}
W = {('A','C'):1,('A','B'):3,('C','B'):1,('C','D'):3,('B','D'):1,('B','G'):3,('D','G'):1}
h = {'A':3,'B':2,'C':2,'D':1,'G':0}

def astar(s,t):
    pq=[(h[s],0,s,None)] 
    parent={}; best_g={s:0}
    while pq:
        f,g,u,p = heapq.heappop(pq)
        if u in parent:   
            continue
        parent[u]=p
        if u==t: break
        for v in G[u]:
            g2 = g + W[(u,v)]
            if v not in best_g or g2 < best_g[v]:
                best_g[v] = g2
                heapq.heappush(pq,(g2 + h[v], g2, v, u))
    # rebuild path
    path=[]; u=t
    while u is not None: path.append(u); u=parent.get(u)
    return path[::-1], best_g[t]

print(astar('A','G'))  # (['A','C','B','D','G'], 4)

## A* Search Algorithm

A* combines the advantages of both UCS and greedy best-first search. It uses a heuristic function to guide the search toward the goal while still guaranteeing optimality (when using an admissible heuristic).

**Characteristics:**
- **Completeness:** Yes - will find a solution if one exists
- **Optimality:** Yes - finds the lowest-cost path (when using an admissible heuristic)
- **Time Complexity:** O(b^d) in the worst case, but typically much better due to the heuristic
- **Space Complexity:** O(b^d) as it must store all nodes in the frontier

**Algorithm Steps:**
1. Initialize a priority queue with the start node (priority = g(start) + h(start))
2. Until the queue is empty or goal is found:
   - Dequeue the node with lowest f-value where f(n) = g(n) + h(n)
   - If it's the goal node, reconstruct the path and return it
   - Otherwise, enqueue all neighbors with their f-values as priority

**Key Components:**
- g(n): Actual cost from start to node n
- h(n): Estimated cost from node n to goal
- f(n): Total estimated cost = g(n) + h(n)

**Evaluation Function Table:**
| Node | g(n) from A | h(n) | f(n) = g(n) + h(n) |
|------|-------------|------|---------------------|
| A    | 0           | 3    | 3                   |
| C    | 1           | 2    | 3                   |
| B    | 2           | 2    | 4                   |
| D    | 3           | 1    | 4                   |
| G    | 4           | 0    | 4                   |

A* will find the optimal path A→C→B→D→G with total cost 4

## Algorithm Comparison

| Algorithm | Data Structure | Complete | Optimal | Time Complexity | Space Complexity | Use Cases |
|-----------|---------------|----------|---------|----------------|-----------------|-----------|
| BFS       | Queue (FIFO)  | Yes      | Yes*    | O(b^d)         | O(b^d)          | Finding shortest path (equal edge cost) |
| DFS       | Stack (LIFO)/Recursion | Yes† | No | O(b^m)         | O(m)            | Path finding with limited memory, maze solving |
| UCS       | Priority Queue | Yes     | Yes     | O(b^(C*/ε))    | O(b^(C*/ε))     | Finding lowest-cost path |
| A*        | Priority Queue | Yes     | Yes‡    | O(b^d)         | O(b^d)          | Informed search with heuristic knowledge |

*BFS is optimal only when all edge costs are equal.  
†DFS is complete only on finite graphs with no cycles.  
‡A* is optimal when the heuristic is admissible.

**Key Terms:**
- b: branching factor
- d: depth of the shallowest solution
- m: maximum depth of the search tree
- C*: cost of the optimal solution
- ε: minimum edge cost

## Conclusion

Choosing the right search algorithm depends on the specific problem characteristics:

1. **BFS** is ideal when the shortest path (in terms of number of edges) is needed and memory is not a constraint.
2. **DFS** is useful when memory is limited or when solutions are likely to be found deep in the tree.
3. **UCS** guarantees the lowest-cost path when edge costs vary.
4. **A*** is the most efficient when a good heuristic is available, combining UCS's optimality with greedy search's efficiency.